In [1]:
import pandas as pd
import geopandas as gpd
from tqdm import tqdm
import hypertools as hyp
import time, glob, itertools

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style='ticks', context='talk')
%matplotlib inline

In [2]:
data=pd.read_csv(r'J:\Geology\WSGS\Projects\Critical Minerals\hu_12_formations_points.csv')

C:\Users\jesse.pisel\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (69) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Calculating Probabilities

In [ ]:
'''
number of counts of a formation where the element value is in some range
divde that number by the total count of the formation = P(Formation|Element)

number of counts of element in some range divided by total number of counts P(Element)

number of counts of formation divided by total number of counts P(Formation)
'''

In [11]:
import itertools
a = data.columns[0:50]
b = data.columns[80:]
b1 = data.HUC_12.unique()
c = list(itertools.product(a, b, b1)) #this creates all possible combinations of huc, formation, and element

In [56]:
d3 = data[data['HUC_12']==c[150000][2]][[c[150000][0], c[150000][1]]]
d3[ (d3.iloc[0:,0].between(bins[2],bins[2+1], 
                            inclusive=True))].iloc[0,1]

0.0

In [57]:
import time
start=time.time()

chunk=np.linspace(20000000,21000000,5) #creates chunks to run in parallel
for k in range(0,4):
        
    fullDF=pd.DataFrame(columns=['bins_ppm', 'formation', 'HUC_12', 'prob', 'element'])

    for i in tqdm(range(chunk[k].astype(int), chunk[k+1].astype(int))):
        pre=[]
        d3=data[data['HUC_12']==c[i][2]][[c[i][0], c[i][1]]] #creates the dataframe for each combination
        bins=np.percentile(data[c[i][0]], [25, 50, 75, 100]) #creates the bins for each element based on quantiles
        testDF = pd.DataFrame()

        if d3.iloc[0:,1].sum()>0:
            for j in range(len(bins)): #for each bin and combination calculate the RHS of bayes theorem
                try:
                    #count the number of formations in HUC
                    #given element bins divided by total number of formations with element bins across the state
                    FmEl = d3[(d3.iloc[0:,1]>0) & (d3.iloc[0:,0].between(bins[j],bins[j+1], 
                            inclusive=True))].iloc[0,1]/d3[d3.iloc[0:,0].between(bins[j],bins[j+1], inclusive=True)].iloc[0,1]
                   #the number of element in the bin in the huc divided by the total count of each element
                    El = d3[d3.iloc[0:,0].between(bins[j],bins[j+1], inclusive=True)].shape[0]/d3.shape[0]
                    #the number of formations in the huc divided by the total count of formations
                    Fm = d3[(d3.iloc[0:,1]>0)].shape[0]/d3.shape[0]
                    pre.append((FmEl*El)/Fm)
                except:
                    pre.append(0)
            testDF = pd.DataFrame() #create a new data frame with bins, formation, HUC, probability, and elements
            testDF['bins_ppm'] = bins
            testDF['formation'] = d3.columns[1]
            testDF['HUC_12'] = c[i][2]
            testDF['HUC_12']=testDF['HUC_12'].astype(str)
            testDF['prob'] = pre
            testDF['element'] = c[i][0]
        else:
            pass
        fullDF = pd.concat([fullDF, testDF])
    file_name = str(chunk[k+1].astype(int))+"_redo.csv"
    DF2=fullDF.dropna()
    DF2.to_csv(r'J:\\Geology\\WSGS\\Projects\\Critical Minerals\\probability maps\\'+file_name)

    
end=time.time()
print(end-start)

 39%|█████████████████████████████                                             | 98177/250000 [06:18<09:45, 259.09it/s]

KeyboardInterrupt: 

 39%|█████████████████████████████                                             | 98177/250000 [06:30<10:03, 251.66it/s]